In [33]:
import configparser
from datetime import datetime, timedelta
import os
from signal import signal, SIGPIPE, SIG_DFL
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, from_unixtime, monotonically_increasing_id, to_date
from pyspark.sql.types import *
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def create_spark_session():
    """
    Purpose:
        Build an access spark session for dealing data ETL of Data Lake
    :return: spark session
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    # .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    # .config("spark.hadoop.fs.s3a.awsAccessKeyId", os.environ['AWS_ACCESS_KEY_ID']) \
    # .config("spark.hadoop.fs.s3a.awsSecretAccessKey", os.environ['AWS_SECRET_ACCESS_KEY'])

    return spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = create_spark_session()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Check data quality:
1. Each dimensional table data schema matches data model
2. There are no empty data
3. Source/ count checks to ensure completeness (It's totally matched with ETL processed data.)

In [23]:
# Dest AWS S3 Bucket
dest_aws_s3_bucket = 'destetlbucket'

source_aws_s3_bucket = "mydatapool/data/"

# Immigration data after ETL processed and resaved in AWS S3
# Expection Count
imm_personal = '3,096,313'
imm_main = '3,096,313'

# Mapping Table
# Expection Count
imm_addr = 55
imm_visa = 3
imm_mode = 4
imm_port = 596
imm_cit_res = 7

# News Article after ETL processed and resaved in AWS S3
# Expection Count
news = '45,827'

# Us Cities Demographics after ETL processed and resaved in AWS S3
# Expection Count
Us_city_demo = '2,891'


dim_bucket_path = S3Path(f"/{dest_aws_s3_bucket}/dimension_table")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
for path in dim_bucket_path.iterdir():
    if path.is_dir():
        path = str(path)
        df = spark.read.parquet(f"s3:/{path}")
        if df.count() <= 0:
            print("Table is empty")
        else:
            print(f"Table Name: {str(path).split('/')[-1]}, Total Rows: {df.count():,}, Total Columns: {len(df.dtypes)}")
            df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Table Name: df_immigration_personal, Total Rows: 3,096,313, Total Columns: 4
root
 |-- imm_per_cic_id: string (nullable = true)
 |-- imm_person_gender: string (nullable = true)
 |-- imm_visatype: string (nullable = true)
 |-- imm_person_birth_year: integer (nullable = true)

Table Name: imm_address, Total Rows: 55, Total Columns: 2
root
 |-- code_of_imm_address: string (nullable = true)
 |-- value_of_imm_address: string (nullable = true)

Table Name: imm_city_res_label, Total Rows: 7, Total Columns: 3
root
 |-- col_of_imm_cntyl: integer (nullable = true)
 |-- value_of_imm_cntyl: string (nullable = true)
 |-- value_of_imm_cntyl_organizations: string (nullable = true)

Table Name: imm_destination_city, Total Rows: 596, Total Columns: 3
root
 |-- code_of_imm_destination_city: string (nullable = true)
 |-- value_of_imm_destination_city: string (nullable = true)
 |-- value_of_alias_imm_destination_city: string (nullable = true)

Table Name: imm_travel_code, Total Rows: 4, Total Columns: 2
r